In [1]:
from datetime import datetime
import sys

sys.path.append('../embeddings/')
sys.path.append('./embeddings/')
sys.path.append('C:/Users/Maods/Downloads/terumo-seg-esclerose-main/terumo-seg-esclerose-main/terumo_seg_esclerose/')
sys.path.append('..')

from dataset import ImageDataLoader
from torch.utils.data import DataLoader
from utils import slice_image_paths
import matplotlib.pyplot as plt
import pickle
import torch
import torchvision
from efficientnet_pytorch import EfficientNet
import numpy as np
import re
from PIL import Image
from torch import nn
from torchvision import transforms

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device
# from terumo_seg_esclerose.cli import run_predict

device(type='cpu')

In [11]:
import typer
from terumo_seg_esclerose.utils.config import ConfigLoaded

from terumo_seg_esclerose.predict.seg_glomerulus import predict as predict_glo
from terumo_seg_esclerose.predict.seg_sclerosis import predict as predict_sle
from mmengine.config import Config
import numpy as np
app = typer.Typer()

path = 'C:/Users/Maods/Downloads/terumo-seg-esclerose-main/terumo-seg-esclerose-main/configs/tiny/tiny-efficientnetb0-unet-pipeline.py'
ConfigLoaded().load_config(path)


Config (path: C:/Users/Maods/Downloads/terumo-seg-esclerose-main/terumo-seg-esclerose-main/configs/tiny/tiny-efficientnetb0-unet-pipeline.py): {'dataset_name': 'hubmap', 'path': './dist/datasets/hubmap/', 'cache': True, 'batch_size': 8, 'multiplier_bin': 4, 'binned_max': 20, 'output_pre': './dist/datasets/hubmap/patch/', 'split': 'train', 'dataset_pre_processing': {'shift_list': [0, 512], 'tile_size': 1024}, 'train_param': {'type': 'train_param', 'output_path': './dist/pretraining/', 'data_csv_path': './dist/datasets/hubmap/patch/data_balanced.csv', 'test_size': 0.3, 'random_state': 19, 'mean': [0.485, 0.456, 0.406], 'std': [0.229, 0.224, 0.225], 'batch_size': 12, 'shuffle': True, 'encoder_name': 'efficientnet-b0', 'encoder_weights': 'imagenet', 'classes': 1, 'activation': None, 'network_name': 'unet', 'loop_param': {'type': 'loop_param', 'filename_checkpoint': 'best_checkpoint.pth', 'cache_weight': None, 'result_csv': 'history.csv', 'max_lr': 0.001, 'epochs': 50, 'weight_decay': 1e-05

In [3]:
def load_model_weights(model, model_path):
    checkpoint = torch.load(model_path)
    model.load_state_dict(checkpoint['state_dict'])
    return model

def get_all_image_files(pathlib_root_folder):
    img_extensions = ['.jpg', '.jpeg', '.png', '.bmp', '.gif']
    img_regex = re.compile('|'.join(img_extensions), re.IGNORECASE)
    image_files = [f for f in pathlib_root_folder.glob('**/*') if f.is_file() and img_regex.search(f.suffix)]
    return image_files


def predict(model, image):
    model.eval()

    with torch.no_grad():
        
        output = model(image.to(device))
        scores = torch.sigmoid(output)
        predictions = (scores>0.5).float()
        _, pred = torch.min(predictions, 1)

    return pred.item()

def sclerosis_predict(image_path):
    mask_glo = predict_glo(image_path)
    mask_sle = predict_sle(image_path)

    inter = np.logical_and(mask_glo > 0.5, mask_sle > 0.5)
    p = np.sum(inter) / (np.sum(mask_glo > 0.5) + 0.00001)

    # print(f"Glomerulu with {p} sclerosis")
    return p

In [4]:

class Net(nn.Module):
    def __init__(self, net_version, num_classes, freeze: bool = False):
        super(Net, self).__init__()
        self.backbone = EfficientNet.from_pretrained('efficientnet-'+net_version)
        self.backbone._fc = nn.Sequential(
            nn.Linear(1280, num_classes),
        )
        if freeze:
            # freeze backbone layers
            for name, param in self.backbone.named_parameters():
                if not name.startswith("_fc"):
                    param.requires_grad = False

    def forward(self, x):
        return self.backbone(x)

In [5]:
hiper_path = './Hypercellularity-2024-02-10 21_19_55.851047-3_fold_min_loss_checkpoint.pth.tar'
membran_path = './Membranous-2024-02-10 21_20_09.125156-1_fold_min_loss_checkpoint.pth.tar'
sclero_path = './Sclerosis-2024-02-10 21_20_15.760971-3_fold_min_loss_checkpoint.pth.tar'
normal_path = './Normal-2024-02-10 21_20_02.347231-2_fold_min_loss_checkpoint.pth.tar'
podoc_path = './Podocytopathy-2024-02-10 21_20_29.072759-2_fold_min_loss_checkpoint.pth.tar'
cresc_path = './Crescent-2024-02-10 21_20_22.470548-1_fold_min_loss_checkpoint.pth.tar'


hiper_model = load_model_weights(Net(net_version="b0", num_classes=2).to(device), hiper_path)
membran_model = load_model_weights(Net(net_version="b0", num_classes=2).to(device), membran_path)
sclero_model = load_model_weights(Net(net_version="b0", num_classes=2).to(device), sclero_path)
normal_model = load_model_weights(Net(net_version="b0", num_classes=2).to(device), normal_path)
podoc_model = load_model_weights(Net(net_version="b0", num_classes=2).to(device), podoc_path)
cresc_model = load_model_weights(Net(net_version="b0", num_classes=2).to(device), cresc_path)


transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
    ])


models = [hiper_model,membran_model,sclero_model,normal_model,podoc_model,cresc_model]

Loaded pretrained weights for efficientnet-b0


RuntimeError: Attempting to deserialize object on a CUDA device but torch.cuda.is_available() is False. If you are running on a CPU-only machine, please use torch.load with map_location=torch.device('cpu') to map your storages to the CPU.

In [21]:
# image = Image.open('C:/Users/Maods/Documents/Development/Mestrado/terumo/apps/terumo-model-binary-glomerulus-hypercellularity/data/raw/hipercellularity/AZAN/PSHIPERCELULARIDADE20200802-10.JPG')
# image = Image.open('C:/Users/Maods/Documents/Development/Mestrado/terumo/apps/terumo-model-binary-glomerulus-hypercellularity/data/raw/crescentes/H_E/FIOCRUZ20190122 (6).jpg')
# image = transform(image)
# image = image.unsqueeze(0)
# with torch.no_grad():
#     cresc_model.eval()
#     output = cresc_model(image.to(device))
#     scores = torch.sigmoid(output)
# scores[:, 0]
# predictions = (scores>0.5).float()
# predictions

tensor([0.7769], device='cuda:0')

In [31]:
data = ImageDataLoader('../data/02_data_split/train_data')
dataloader = DataLoader(data.dataset, batch_size=128, shuffle=False)

target = []
paths = []
labels = []
num_att = 6
feature_embeddings = np.empty((0, num_att))




for i, (x, y, path, label) in enumerate(dataloader):
    x = x.to(device=device)
    with torch.no_grad():
        prediction_columns = []
        for model in models:
            model.eval()
            output = model(x)
            scores = torch.sigmoid(output)
            prediction_columns.append(scores[:, 0].view(-1, 1))
        

    prediction_matrix = torch.cat(prediction_columns, dim=1)
    feature_embeddings = np.vstack((feature_embeddings, prediction_matrix.cpu().detach().numpy()))

    target.extend(list(y.cpu().detach().numpy()))
    paths.extend(slice_image_paths(path))
    labels.extend(label)


data_dict = {
    "model": 'semantic_att',
    "embedding":feature_embeddings,
    "target":target,
    "paths": paths,
    "classes":labels
}



with open(f'./semantic_train.pickle', 'wb') as pickle_file:
    pickle.dump(data_dict, pickle_file)

In [30]:

data = ImageDataLoader('../data/03_test')
dataloader = DataLoader(data.dataset, batch_size=50, shuffle=False)

target = []
paths = []
labels = []
num_att = 6
feature_embeddings = np.empty((0, num_att))




for i, (x, y, path, label) in enumerate(dataloader):
    x = x.to(device=device)
    with torch.no_grad():
        prediction_columns = []
        for model in models:
            model.eval()
            output = model(x)
            scores = torch.sigmoid(output)
            prediction_columns.append(scores[:, 0].view(-1, 1))


    prediction_matrix = torch.cat(prediction_columns, dim=1)
    feature_embeddings = np.vstack((feature_embeddings, prediction_matrix.cpu().detach().numpy()))

    target.extend(list(y.cpu().detach().numpy()))
    paths.extend(slice_image_paths(path))
    labels.extend(label)


data_dict = {
    "model": 'semantic_att',
    "embedding":feature_embeddings,
    "target":target,
    "paths": paths,
    "classes":labels
}



with open(f'./semantic_test.pickle', 'wb') as pickle_file:
    pickle.dump(data_dict, pickle_file)

In [27]:
feature_embeddings[0]

array([0.21664597, 0.05527985, 0.0024234 , 0.01381866, 0.04079605,
       0.95408595])

['03_test/crescentes/312969_3_En_20_Fig3_HTML.png',
 '03_test/crescentes/BtkTpI4CMAEee6d.jpg',
 '03_test/crescentes/Fig-3-The-histology-of-the-kidney-A-glomerulus-showing-accumulation-of-cells-and.png',
 '03_test/crescentes/Glomerulus-showing-crescentic-glomerulonephritis-due-to-pauci-immune-ANCA-associated_Q320.jpg',
 '03_test/crescentes/H-E-stains-showing-crescentic-glomerulonephritis-with-moderate-interstitial-inflammation.png',
 '03_test/crescentes/Rapidly_progressive_glomerulonephritis_type_3_high.jpg',
 '03_test/crescentes/cellular-crescent-glomerulus-quiz.jpg',
 '03_test/crescentes/ch523.fig3_.jpg',
 '03_test/crescentes/kidneyrpgnMurshed02.jpg',
 '03_test/crescentes/kidneyrpgnMurshed25.jpg',
 '03_test/crescentes/urn_cambridge.org_id_binary_20181009125204364-0075_9781107281981_61398fig7_3.png',
 '03_test/crescentes/urn_cambridge.org_id_binary_20181009125204364-0075_9781107281981_61398fig7_6.png',
 '03_test/hipercellularity/E7yBoUfwoEMqTSCkMhmm1w.jpg',
 '03_test/hipercellularity/F